# Zeroth Order Optimization (ZOO) Attack Experiments - Targeted DoH
## Warning: Before running this notebook, make sure you ran the following steps :
1. Normalize data: [normalize-data.ipynb](./bsides-experiments/normalize-data.ipynb)
2. Train a model: [build_model.ipynb](./bsides-experiments/build_model.ipynb)
3. Get DoH tunnel tools feature limits: [get-doh-tunnel-tool-limits.ipynb](./bsides-experiments/get-doh-tunnel-tool-limits.ipynb)
4. Increase/decrease the feature limits according to your needs: Edit the file `dnstt-limits-prod.csv`
5. Choose features to attack: Edit the array `features_to_attack` on this notebook 

In [ ]:
import numpy as np
import pandas as pd

from joblib import dump, load

#!pip install adversarial-robustness-toolbox
from art.estimators.classification import SklearnClassifier

# Loading Dataset

In [ ]:
input_dir = '../datasets/'

In [ ]:
X_features_dnstt_file = input_dir + '27072024-tunnel.csv'

In [ ]:
from sklearn.preprocessing import normalize

# Load the X_features 
X_features_dnstt = pd.read_csv(X_features_dnstt_file, sep=',')

# concat all features
X = pd.concat([X_features_dnstt])
X = X.drop(columns=['SourceIP', 'DestinationIP', 'TimeStamp', 'SourcePort', 'DestinationPort', 'Duration', 'DoH'])

# Set display option to show all columns
#pd.set_option('display.max_columns', None)
#from IPython.display import display
display(X.head())

In [ ]:
# Normalize the features
# We got the l2_norms to unnormalize the features after the attack
l2_norms = np.linalg.norm(X, axis=1, keepdims=True)
X = pd.DataFrame(normalize(X, norm='l2', axis=1), columns=X.columns)
display(X.head())

# Attacking

In [ ]:
model_input_dir = './'

# Evaluating Constrained Zoo-DoH - DNSTT

In [ ]:
# Load the X_features
# Convert the DataFrame to a NumPy array
X_test = np.array(X)

In [ ]:
display(X_test.shape[0])

In [ ]:
# Load doh tunnel tools feature limits
input_dir = './'
features_limits = pd.read_csv(f'{input_dir}/dnstt-limits-prod.csv', sep=',')
min_val = features_limits['min'].to_numpy()
max_val = features_limits['max'].to_numpy()
clip_values = (min_val, max_val)

In [ ]:
print(clip_values)


# Attacking Target DoH

In [ ]:
import myzoo.target_zoo as zoo_targeted
model_filename = 'GradientBoosting-e-valente-customized.joblib'
model = load(model_input_dir + model_filename)

# Create blackbox object
art_classifier = SklearnClassifier(model=model, clip_values=clip_values)

# Features to attack
# 0 -> FlowBytesSent
# 1 -> FlowSentRate
# 2 -> FlowBytesReceived 
# 3 -> FlowReceivedRate    
# 4 -> PacketLengthVariance
# 5 -> PacketLengthStandardDeviation
# 6 -> PacketLengthMean
# 7 -> PacketLengthMedian
# 8 -> PacketLengthMode  
# 9 -> PacketLengthSkewFromMedian  
# 10 -> PacketLengthSkewFromMode
# 11 -> PacketLengthCoefficientofVariation
# 12 -> PacketTimeVariance
# 13 -> PacketTimeStandardDeviation
# 14 -> PacketTimeMean
# 15 -> PacketTimeMedian
# 16 -> PacketTimeMode
# 17 -> PacketTimeSkewFromMedian
# 18 -> PacketTimeSkewFromMode
# 19 -> PacketTimeCoefficientofVariation
# 20 -> ResponseTimeVariance
# 21 -> ResponseTimeStandardDeviation
# 22 -> ResponseTimeMean
# 23 -> ResponseTimeMedian
# 24 -> ResponseTimeMode
# 25 -> ResponseTimeSkewFromMedian
# 26 -> ResponseTimeSkewFromMode
# 27 -> ResponseTimeCoefficientofVariation
features_to_attack = [0, 1, 4, 5, 6, 12, 13, 14]

# Create ART Zeroth Order Optimization attack
zoo = zoo_targeted.ZooAttack(classifier=art_classifier, confidence=0.0, targeted=True, learning_rate=1e-3, max_iter=30,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_importance=False, nb_parallel=1, 
                        batch_size=1, variable_h=0.2, feature_indices=features_to_attack, verbose=True)

In [ ]:
#attacking
size_data = len(X_test)
x_test_adv = zoo.generate(X_test, np.zeros(size_data, dtype='int'))

In [ ]:
# attack success rate
success = 0
total_to_attack = 0

success_indices = []
for i in range(size_data):
  prediction_before_attack = model.predict(X_test[i].reshape(1, -1))
  prediction_after_attack = model.predict(x_test_adv[i].reshape(1, -1))

  if prediction_before_attack == [1]:
    total_to_attack = total_to_attack + 1

    if prediction_after_attack == [0]:
      success = success + 1
      success_indices.append(i)
  
print(f'total samples to be attacked (malicious samples) {total_to_attack}')

print(f'total success (malicious -> benign) {success}')
print(f'total percent samples successfully attacked {success/total_to_attack}')

In [ ]:
# Get the four indices of the samples that were successfully attacked
for i in success_indices:
    diff = abs(x_test_adv[i] * l2_norms[i])- (X_test[i] * l2_norms[i])
    #print(diff)
    # get the four biggest difference indices
    biggest_indices = np.argsort(diff)[-5:]
    for j in biggest_indices:
        print(f'{X.columns[j]}: {diff[j]}')
    
    print("======")

In [ ]:
# Print Data
for i in success_indices:
    print("Original data: ")
    print(X_test[i] * l2_norms[i])
    print("Attacked Data:")
    print(x_test_adv[i] * l2_norms[i])
    print("-------\n")
 